### Generating data

In [1]:
import torch
from torch import nn
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [2]:
x,y=make_regression(n_samples=500,n_features=8,noise=0.2,random_state=42)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train=torch.from_numpy(x_train).type(torch.float32)
x_test=torch.from_numpy(x_test).type(torch.float32)
y_train=torch.from_numpy(y_train).type(torch.float32)
y_test=torch.from_numpy(y_test).type(torch.float32)

### Model building

In [3]:
class simple_nn(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights=nn.Parameter(torch.zeros(x.shape[1],dtype=torch.float32),requires_grad=True)
        self.bias=nn.Parameter(torch.zeros(1,dtype=torch.float32),requires_grad=True)
    def forward(self,x):
        return torch.matmul(x,self.weights)+self.bias
model=simple_nn()
loss_fn=nn.MSELoss()
optimizer=torch.optim.SGD(lr=0.01,params=model.parameters())

In [4]:
epochs=600
for epoch in range(epochs):
    model.train()
    pred=model(x_train)
    loss=loss_fn(pred,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch%50==0:
        print(f'| Epoch" {epoch} | loss:{loss:.4f}| ')

| Epoch" 0 | loss:22253.3867| 
| Epoch" 50 | loss:3170.3982| 
| Epoch" 100 | loss:465.1332| 
| Epoch" 150 | loss:70.6988| 
| Epoch" 200 | loss:11.2241| 
| Epoch" 250 | loss:1.8929| 
| Epoch" 300 | loss:0.3611| 
| Epoch" 350 | loss:0.0971| 
| Epoch" 400 | loss:0.0493| 
| Epoch" 450 | loss:0.0402| 
| Epoch" 500 | loss:0.0384| 
| Epoch" 550 | loss:0.0381| 


In [5]:
model.state_dict()

OrderedDict([('weights',
              tensor([42.9025,  8.5429, 48.3477, 48.5011, 97.2438, 51.8062, 61.4017, 23.6586])),
             ('bias', tensor([-0.0064]))])

### Model evaluating

In [6]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

model.eval()
with torch.inference_mode():
    pred=model(x_test)

print(f'MSE: {mean_squared_error(pred,y_test)}')
print(f'MAE: {mean_absolute_error(pred,y_test)}')
print(f'Loss: {loss_fn(pred,y_test)}')

MSE: 0.04199009092178258
MAE: 0.15942148089408875
Loss: 0.041990090161561966
